In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 33790287
paper_name = 'yu_dang_2021' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/41467_2021_22257_MOESM3_ESM.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 3437 x 8


In [7]:
original_data.head()

,ORF name,SEBYL LS Score,SEBYl LS rank,MM lifespan,MM wt ls,MM cell counted,Unnamed: 6,"MM lifespan and cell count from ( McCormick,2015)"
0,YKL116C,37.420,1,28.32,25.16,25.0,NaN,NaN
1,YBL060W,36.480,2,30.00,25.60,10.0,NaN,NaN
2,YGL118C,35.130,3,31.80,24.20,5.0,NaN,NaN
3,YOR112W,32.415,4,25.80,27.25,25.0,NaN,NaN
4,YDR142C,30.570,6,30.20,22.80,5.0,NaN,NaN


In [8]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF name, SEBYL LS Score, SEBYl LS rank, MM lifespan, MM wt ls, MM cell counted, Unnamed: 6, MM lifespan and cell count from ( McCormick,2015), orf]
Index: []


In [12]:
original_data['data'] = pd.to_numeric(original_data['SEBYL LS Score'], errors='coerce')

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(3409, 1)

# Prepare the final dataset

In [17]:
data = original_data.copy()

In [18]:
dataset_ids = [21887]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,21887
data_type,value
orf,
YAL002W,18.683333
YAL004W,19.803333
YAL005C,20.525000
YAL007C,19.155000
YAL008W,22.260000


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 18


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21887
,data_type,value
gene_id,orf,
2,YAL002W,18.683333
1863,YAL004W,19.803333
4,YAL005C,20.525000
5,YAL007C,19.155000
6,YAL008W,22.260000


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id           21887          
data_type            value    valuez
gene_id orf                         
2       YAL002W  18.683333 -0.948969
1863    YAL004W  19.803333 -0.421436
4       YAL005C  20.525000 -0.081523
5       YAL007C  19.155000 -0.726809
6       YAL008W  22.260000  0.735682

# Print out

In [26]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [27]:
from IO.save_data_to_db3 import *

In [28]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 33790287...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.60s/it]

Updating the data_modified_on field...
